In [1]:
import numpy as np
import kirchhoff.circuit_init as kfi
from goflow.adapter import init_ivp as gi

#initialize circuit+flow pattern
# C = kfi.initialize_circuit_from_crystal('hexagonal',2)
C = kfi.initialize_circuit_from_crystal('square',3)

# set model and model parameters
pars_src = {
    'modeSRC': 'dipole_border',
}
pars_plx = {
    'modePLX':'default',
}
pars_model = {
    'phi0': .5,
    'alpha_0': 100.,
    'alpha_1': 100.,
}

# # initialize dynamic system and set integration parameters
morpheus = gi.morph_dynamic(C, 'link', [pars_model, pars_src, pars_plx]) 

Set source: dipole_border
set_source_landscape(): dipole_border is set and consistent :)
Set plexus: default
set_plexus_landscape(): default is set and consistent :)


In [ ]:
# numerically evaluate the system
cnd = morpheus.flow.circuit.edges['conductivity']
cnd_scale = morpheus.flow.circuit.scales['conductance']
sp = {
    't0': 0.,
    't1': 3.,
    'x0': np.power(cnd/cnd_scale,0.25),
}
nsol = morpheus.autoSolve((sp['t0'],sp['t1']), sp['x0'])
print(nsol)

In [ ]:
dataPoints = zip(nsol.t,nsol.y.transpose())
Model = morpheus.model
args = Model.solver_options['args']
cost = [Model.calc_cost_system(t, y, *args) for t, y in dataPoints]
# # # plot dynamic data such as radii and costs
import matplotlib.pyplot as plt
fig,axs=plt.subplots(2,1,figsize=(12,6),sharex=True)
axs[0].plot(nsol.t,nsol.y.transpose(),alpha=0.1,color='b')
axs[1].plot(nsol.t,cost,alpha=0.2,color='b')

for i in range(2):
    axs[i].grid(True)
    
axs[1].set_xlabel(r'time $t$')
axs[0].set_ylabel(r'radii $r$')
axs[1].set_ylabel(r'metabolic cost $\Gamma$')
axs[1].set_yscale('log')
plt.show()

In [ ]:
# plot network with data of choice
morpheus.flow.circuit.edges['conductivity'] = nsol.y.transpose()[-1]
fig=morpheus.flow.circuit.plot_circuit(linewidth = [morpheus.flow.circuit.edges['conductivity']*10.])
fig.show()